# Starcraft 2 Search Tree Algorithm Project - Adam O'Mahony

The idea behind this algorithm project is to simulate an RTS game into a turn based game board.
The game revolves around two opposing forces, the Zergs and the Marines, while the actual game has multiple unit types and various costs associated with each unit.

For this game we will use the most basic unit from each team, the Marine and the Zergling. The base cost of a marine is 1 supply and 50 minerals, the base cost of a Zergling is .5 supply and 25 minerals, as such 1 marine equates to 2 Zerglins in terms of cost of units.

https://www.unitstatistics.com/starcraft2/ was used for getting base unit Hp, Dps etc.

Based on the data the following information was used for the units:

| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |



The core concept of this approach is to test whether the search tree algorithm can be used to effectively force a draw or win for the team chosen.

In [10]:
from IPython.display import Markdown

table = """
| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |
"""

display(Markdown(table))


| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |


# Current Board

The board for the game is variable, it has been set up as a default as 30 wide, 10 high, with a marine count of 3 and zergling count of 6. Map X, Y, number of marines, number of zergs.

The initial board would be set up based on the number of zergs, marines and the size of the board, however there was an issue where instead of the zergs and marines spawning on individual cells it was putting them on top of eachother, so I had to restrict this.

Each turn of the game invloves all units making an action, movement, retreat, or attack. The first of these was movement and retreat.

The initial plan was for the units to have movement in any direction based off their move-speed, however this was too larger a branching factor and as such it was reduced to forward movement and backwards movement (Retreat) however this was still too much and the final approach was to have forward movement as standard and the retreat movement in certain scenarios only.
Marines since there is less of them have the retreat movement available all the time and reduced movement when an enemy is within attack range.

![image](./images/movement.png)

Movement is also restricted to if the cell is free, there was also an issue with the Zergling units not behaving as expected due to the need to have the enemy unit be exactly their movement speed away, this was resolved by adding a charging movement to melee units (uses units with attack range of 1).

Another issue with movement was that if a zerg got behind the marines or a marine got behind the zerglings, there was no possible moves other than retreat to get in range. As such the forward movement was changed to be based on direction of nearest enemy. This function first used Manhattan distance, later Euclidean distance was implemented throughout the project.

A final issue occured that when there was only 1 unit left in each team, they would only have the possiblity to retreat but the unit might be above or below them. as such a case for when a unit is the last on their team they get the initial omnidirectional of movement.

Attacks have two modes, Melee attacks and Ranged attacks, Melee units have the charging feature above that allows them to get into melee range, any unit can attack in melee range, including ranged units. If a unit is in melee range it cannot retreat and must attack.

Ranged units can attack enemies in range, first Manhatthan distance was used but to better get the diagonal attack possibilities, Euclidean distance was used instead. When a unit is in attack range it is added to the attackable targets for that unit.

When a units hp reaches 0 it is removed from the board. One issue that came up with the unit HP is that due to the generation of children with deepcopy, the boards units are copied and as such the unit couldn't be retreived directly from the attack target, as such the targets x and y was used to apply the damage to th correct unit at that time.

The evaluation of the current state of the board was crucial for this project, this evaluation is based on two things, unit HP and distance from enemy units.
These two parameters are added as variables to the current board: health_weight, positioning_weight.

This is evaluated for each unit and thus a team score is created, the zerg score is taken away from the marine score, thus a high value is beneficial to the marines and a lower value is beneficial to the zergs.

This evaluation function is used in the min_max function to find the next best move for the current team making a move.

There were some issues with the development of this function, at first I had another variable of potential attack, this was removed since it wasn't positively effecting the evaluation. I also considered tracking the % of initial units, but this would invlove tracking the starting units given at the begining of the game, and would add complexity with child generation.

Another issue I had was that during the evaluation of the positioning, I originally tryed to make it based on position to the centre of the board, however this sometimes gave a divison by zero error if the unit was in the centre.

![image](./images/division_error.png)

This was replaced with manhattan distance to enemy units, and then this was replaced with Euclidean distance in the final itteration.

The branching factor for this project was quite large since each unit has multiple possible actions, particularly at the begining when the units are all alive and none are in combat range since combat reduces the possible moves the initial stages of the game require a large branching factor.

The possible moves per turn are culmiated in the function get_team_possible_actions() which returns all the possible moves for the team for that turn. This went through some itterations as well but the main feature of it was to ensure that unique scenarios were created, to do so I used the line:
        return list(set(tuple(combo) for combo in product(*team_unit_actions)))
This requires the import of: from itertools import product

The state of play for this game board is straight forward as it evaluates the ammount of units remaining for each side only. As such the loop for the play game function has a max itterations to prevent infinite looping.


In [11]:
import copy
from itertools import product

In [12]:
class CurrentBoard:
    def __init__(self, width=30, height=10, marine_count=3, zergling_count=6, hp_weight=0.5,position_weight=0.3):
        self.width = width
        self.height = height
        self.board = [[None for _ in range(width)] for _ in range(height)]
        self.marines = []  # List to store Marine objects
        self.zerglings = []  # List to store Zergling objects
        self.center_x = width // 2
        self.center_y = height // 2
        self.marine_count = marine_count
        self.zergling_count = zergling_count
        self.state = self.state_of_board()
        self.health_weight = hp_weight
        self.positioning_weight = position_weight


    def is_in_melee_combat(self, unit):
        # Check if unit is adjacent to an enemy unit
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'

        # Check adjacent squares
        directions = [
            (0, 1),  # Up
            (0, -1),  # Down
            (1, 0),  # Right
            (-1, 0),  # Left
            (1, 1),  # Up-Right
            (1, -1),  # Down-Right
            (-1, 1),  # Up-Left
            (-1, -1)  # Down-Left
        ]

        for dx, dy in directions:
            check_x = unit.x + dx
            check_y = unit.y + dy

            # Check if the adjacent square is within board bounds
            if (0 <= check_x < self.width and
                    0 <= check_y < self.height):
                adjacent_unit = self.board[check_y][check_x]
                # If there's an enemy unit in an adjacent square
                if (adjacent_unit is not None and
                        adjacent_unit.team == enemy_team):
                    return True

        return False

    def get_targets_in_melee_range(self, unit):
        """
        Check if unit is adjacent to an enemy unit and returns targets in melee range
        """
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'
        enemy_units = self.zerglings if unit.team == 'Marine' else self.marines
        targets_in_melee = []

        # Check adjacent squares
        directions = [
            (0, 1),  # Up
            (0, -1),  # Down
            (1, 0),  # Right
            (-1, 0),  # Left
            (1, 1),  # Up-Right
            (1, -1),  # Down-Right
            (-1, 1),  # Up-Left
            (-1, -1)  # Down-Left
        ]

        for dx, dy in directions:
            check_x = unit.x + dx
            check_y = unit.y + dy

            # Check if the adjacent square is within board bounds
            if (0 <= check_x < self.width and
                    0 <= check_y < self.height):
                adjacent_unit = self.board[check_y][check_x]
                # If there's an enemy unit in an adjacent square
                if (adjacent_unit is not None and
                        adjacent_unit.team == enemy_team):
                    targets_in_melee.append(adjacent_unit)

        return targets_in_melee


    def get_attackable_targets(self, unit):
        # was previously using manhatthan distance
        targets = []
        # Determine targets based on unit type and attack range
        enemy_list = self.zerglings if unit.team == 'Marine' else self.marines
        for potential_target in enemy_list:
            # Calculate Euclidean distance
            distance = ((unit.x - potential_target.x) ** 2 + (
                        unit.y - potential_target.y) ** 2) ** 0.5  # Euclidean Distance
            if distance <= unit.attack_range:
                targets.append(potential_target)

        return targets


    def get_possible_actions(self, unit):
        # Check if unit can attack and is not in melee combat (indicating a ranged unit)
        attackable_targets = self.get_attackable_targets(unit)
        #check if in melee
        in_melee_combat = self.is_in_melee_combat(unit)
        # Get possible moves
        possible_moves = self.get_possible_moves(unit)
        # Retreat actions:
        retreat_directions = self.get_retreat_directions(unit)

        # Detailed action generation logic
        actions = []

        # If in melee combat, prioritize attack
        if in_melee_combat:
            melee_targets = self.get_targets_in_melee_range(unit)
            attack_actions = [('attack', target) for target in melee_targets]
            actions.extend(attack_actions)

        # First, check for attack actions
        if attackable_targets and not in_melee_combat:
            attack_actions = [('attack', target) for target in attackable_targets]
            retreat_actions = [('retreat', direction) for direction in retreat_directions]
            actions.extend(attack_actions)
            actions.extend(retreat_actions)


        # Add move actions if no attacks or not in melee
        if not actions:
            move_actions = [('move', move) for move in possible_moves]
            actions.extend(move_actions)

        # Add retreat actions if no other actions
        if not actions:
            retreat_actions = [('retreat', direction) for direction in retreat_directions]
            actions.extend(retreat_actions)


        return actions


    def get_possible_moves(self, unit):
        """
        Generate possible moves dynamically, with omnidirectional movement for last unit.
        """
        moves = []
        charge_moves = []

        # Determine primary movement direction based on enemy position
        nearest_enemy = self.find_nearest_enemy(unit)

        if nearest_enemy is None:
            # If no enemy found, return current position
            return [(unit.x, unit.y)]

        dx = 1 if nearest_enemy.x > unit.x else -1
        dy = 1 if nearest_enemy.y > unit.y else -1

        # Check if enemy is within charging range and unit is a melee unit (attack_range of 1)
        distance_to_enemy = abs(unit.x - nearest_enemy.x) + abs(unit.y - nearest_enemy.y)
        if unit.attack_range == 1 and distance_to_enemy <= unit.speed:  # Charging distance check
            # Calculate the charge move (one space in front of enemy)
            charge_x = nearest_enemy.x - dx  # One space in front of the enemy
            charge_y = nearest_enemy.y - dy

            # Check if the charge position is within the board boundaries
            if 0 <= charge_x < self.width and 0 <= charge_y < self.height:
                # Check if the charge position is unoccupied
                if self.board[charge_y][charge_x] is None:
                    charge_moves.append((charge_x, charge_y))
                else:  # Find alternative Charge Move using diagnols
                    # Calculate distances to upper-left and bottom-left
                    ul_x = nearest_enemy.x - 1
                    ul_y = nearest_enemy.y - 1
                    bl_x = nearest_enemy.x - 1
                    bl_y = nearest_enemy.y + 1

                    # Verify if they are free, if they are choose the closest one
                    if 0 <= ul_x < self.width and 0 <= ul_y < self.height and self.board[ul_y][ul_x] is None:
                        ul_dist = abs(unit.x - ul_x) + abs(unit.y - ul_y)
                        charge_moves.append((ul_x, ul_y))
                    if 0 <= bl_x < self.width and 0 <= bl_y < self.height and self.board[bl_y][bl_x] is None:
                        bl_dist = abs(unit.x - bl_x) + abs(unit.y - bl_y)
                        charge_moves.append((bl_x, bl_y))

            # If there is a valid charge move, return it immediately.
            if charge_moves:
                return charge_moves

        # Check if the current team has only one unit left
        team_units = self.marines if unit.team == 'Marine' else self.zerglings
        is_last_unit = len(team_units) == 1

        # Generate omnidirectional moves if it's the last unit
        if is_last_unit:
            directions = [
                (1, 0),  # Right
                (-1, 0),  # Left
                (0, 1),  # Up
                (0, -1),  # Down
                (1, 1),  # Up-Right
                (1, -1),  # Down-Right
                (-1, 1),  # Up-Left
                (-1, -1)  # Down-Left
            ]
        elif unit.team == 'Zergling':
            directions = [
                (dx, 0),  # Primary horizontal movement towards enemy
                (dx, dy),  # Diagonal movement towards enemy
                (dx, -dy),  # Alternate diagonal
                # (-dx, 0)  # Backward movement if needed
            ]
        else:  # Marine
            directions = [
                (-dx, 0),  # Primary horizontal movement towards enemy (reversed for Marines)
                (-dx, dy),  # Diagonal movement towards enemy
                (-dx, -dy),  # Alternate diagonal
                (dx, 0)  # Backward movement if needed
            ]

        for move_dx, move_dy in directions:
            # Calculate end position based on unit's full speed
            move_x = unit.x + move_dx * unit.speed
            move_y = unit.y + move_dy * unit.speed

            # Check if move is within board boundaries
            if 0 <= move_x < self.width and 0 <= move_y < self.height:
                # Get the unit at the destination tile
                dest_unit = self.board[move_y][move_x]

                # If destination is occupied
                if dest_unit is not None:
                    # Check if the occupying unit is an enemy
                    if dest_unit.team != unit.team:
                        # Calculate the tile just before the occupied tile
                        pre_move_x = unit.x + move_dx * (unit.speed - 1)
                        pre_move_y = unit.y + move_dy * (unit.speed - 1)

                        # Check if pre-move tile is within bounds and unoccupied
                        if (0 <= pre_move_x < self.width and
                                0 <= pre_move_y < self.height and
                                self.board[pre_move_y][pre_move_x] is None):
                            moves.append((pre_move_x, pre_move_y))
                else:
                    # Destination is unoccupied, add full move
                    moves.append((move_x, move_y))

        if not moves and not charge_moves:
            moves = [(unit.x, unit.y)]
        if charge_moves:
            return charge_moves
        return moves




    def find_nearest_enemy(self, unit):
        """
        changed from Manhattan distance to Euclidean distance
        Find the nearest enemy unit to the given unit using Euclidean distance
        """
        enemy_team = 'Marine' if unit.team == 'Zergling' else 'Zergling'
        enemy_units = self.marines if enemy_team == 'Marine' else self.zerglings

        if not enemy_units:
            return None

        # Calculate Euclidean distance to each enemy
        enemies_with_distance = [
            (enemy, ((unit.x - enemy.x) ** 2 + (unit.y - enemy.y) ** 2) ** 0.5)
            for enemy in enemy_units
        ]

        # Find the enemy with the minimum distance
        return min(enemies_with_distance, key=lambda x: x[1])[0]




    def get_retreat_directions(self, unit):
        """
        Generate possible retreat moves with:
        - Unit-specific directional restrictions
        - Board boundary checks
        - Occupied tile checks
        - stay in place option
        """
        retreats = [(unit.x, unit.y)]


        # Determine retreat direction based on unit type
        if unit.team == 'Zergling':
            dx, dy = -1, 0  # Left (backward)
        else:  # Marine
            dx, dy = 1, 0  # Right (backward)

        # Calculate end retreat position based on unit's full speed
        retreat_x = unit.x + dx * unit.speed
        retreat_y = unit.y + dy * unit.speed

        # Check if retreat is within board boundaries
        if (0 <= retreat_x < self.width and 0 <= retreat_y < self.height):
            # If destination is occupied, check the tile just before
            if self.board[retreat_y][retreat_x] is not None:
                # Calculate the tile just before the occupied tile
                pre_retreat_x = unit.x + dx * (unit.speed - 1)
                pre_retreat_y = unit.y + dy * (unit.speed - 1)

                # Check if pre-retreat tile is within bounds and unoccupied
                if (0 <= pre_retreat_x < self.width and
                        0 <= pre_retreat_y < self.height and
                        self.board[pre_retreat_y][pre_retreat_x] is None):
                    retreats.append((pre_retreat_x, pre_retreat_y))
            else:
                # Destination is unoccupied, add full retreat
                retreats.append((retreat_x, retreat_y))

        return retreats


    def get_team_possible_actions(self, team):
        """
        Generate all possible action combinations for a team
        Ensures every unit takes exactly one action
        """
        # Select units based on team
        units = self.marines if team == 'Marine' else self.zerglings

        # Will store possible actions for each unit
        team_unit_actions = []

        # Generate possible actions for each unit
        for unit in units:
            # Get possible actions for this specific unit
            unit_actions = self.get_possible_actions(unit)
            # If no actions are possible, this is an invalid state
            if not unit_actions:
                if unit.hp > 0:
                    raise ValueError(f"Unit {unit.team} at ({unit.x}, {unit.y}) has no possible actions")

            team_unit_actions.append(unit_actions)

        # Generate unique action combinations
        return list(set(tuple(combo) for combo in product(*team_unit_actions)))


    def setup_initial_board(self):
        """
        Set up initial board with units at the exact map edges
        """
        # Clear existing units
        self.marines.clear()
        self.zerglings.clear()
        self.board = [[None for _ in range(self.width)] for _ in range(self.height)]

        # Calculate vertical center of the board
        center_y = self.height // 2

        # Calculate front and back row counts
        zergling_front_count = self.zergling_count // 2
        zergling_back_count = self.zergling_count - zergling_front_count

        marine_front_count = self.marine_count // 2
        marine_back_count = self.marine_count - marine_front_count

        # Zerglings formation (left edge)
        front_zergling_x = 0  # Leftmost edge of the map
        back_zergling_x = 1  # One step from the left edge

        # Marines formation (right edge)
        front_marine_x = self.width - 1  # Rightmost edge of the map
        back_marine_x = self.width - 2  # One step from the right edge

        # Vertical positioning for Zerglings
        zergling_front_rows = [
            y for y in range(
                center_y - zergling_front_count // 2,
                center_y + (zergling_front_count + 1) // 2
            )
        ]
        zergling_back_rows = [
            y for y in range(
                center_y - zergling_back_count // 2,
                center_y + (zergling_back_count + 1) // 2
            )
        ]

        # Vertical positioning for Marines
        marine_front_rows = [
            y for y in range(
                center_y - marine_front_count // 2,
                center_y + (marine_front_count + 1) // 2
            )
        ]
        marine_back_rows = [
            y for y in range(
                center_y - marine_back_count // 2,
                center_y + (marine_back_count + 1) // 2
            )
        ]

        # Place Zerglings in front rows
        for y in zergling_front_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(front_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][front_zergling_x] = zergling

        # Place Zerglings in back rows
        for y in zergling_back_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(back_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][back_zergling_x] = zergling

        # Place Marines in front rows
        for y in marine_front_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(front_marine_x, y)
                self.marines.append(marine)
                self.board[y][front_marine_x] = marine

        # Place Marines in back rows
        for y in marine_back_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(back_marine_x, y)
                self.marines.append(marine)
                self.board[y][back_marine_x] = marine

        # Verify unit counts
        assert len(self.zerglings) == self.zergling_count, f"Zergling count is {len(self.zerglings)}, should be {self.zergling_count}"
        assert len(self.marines) == self.marine_count, f"Marine count is {len(self.marines)}, should be {self.marine_count}"
        self.state = self.state_of_board()


    def move_unit(self, unit, target):
        # Remove unit from current position
        self.board[unit.y][unit.x] = None

        # Update unit's coordinates
        new_x, new_y = target
        unit.x = new_x
        unit.y = new_y

        # Place unit in new position
        self.board[new_y][new_x] = unit


    def attack_unit(self, attacker, target_unit):
        # Find the actual target unit on the board at the given coordinates
        actual_target = self.board[target_unit.y][target_unit.x]

        if actual_target is None:
            return  # No unit to attack

        # Calculate damage based on attacker's characteristics
        damage = attacker.damage

        # Reduce target's health
        actual_target.hp -= damage

        # Check if target is destroyed
        if actual_target.hp <= 0:
            # Remove target from board and unit list
            self.board[actual_target.y][actual_target.x] = None

            if actual_target.team == 'Marine':
                # Find and remove the unit with matching coordinates and team
                self.marines = [marine for marine in self.marines
                                if not (marine.x == actual_target.x and
                                        marine.y == actual_target.y and
                                        marine.team == actual_target.team)]
            else:
                # Same for Zerglings
                self.zerglings = [zergling for zergling in self.zerglings
                                  if not (zergling.x == actual_target.x and
                                          zergling.y == actual_target.y and
                                          zergling.team == actual_target.team)]



    def display_board(self):
        """
        Visualize the current board state with precise unit placement
        """
        # Create a copy of the board to visualize
        display_grid = [[' .' for _ in range(self.width)] for _ in range(self.height)]

        # Print the board with detailed unit placement
        print("\nDebug - Unit Positions:")
        for zergling in self.zerglings:
            print(f"Zergling: x={zergling.x}, y={zergling.y} hp={zergling.hp}")
            display_grid[zergling.y][zergling.x] = ' Z'

        for marine in self.marines:
            print(f"Marine: x={marine.x}, y={marine.y} hp={marine.hp}")
            display_grid[marine.y][marine.x] = ' M'

        # Print the board
        for row in display_grid:
            print(''.join(row))

        # Print additional unit information
        print(f"\nMarines: {len(self.marines)}, Zerglings: {len(self.zerglings)}")


    def state_of_board(self):
        # Check if all Marines are dead
        if len(self.marines) == 0:
            return "Zergling"

        # Check if all Zerglings are dead
        if len(self.zerglings) == 0:
            return "Marine"

        # Check if mutual destruction.
        if (len(self.marines) + len(self.zerglings) >= self.width * self.height or
                len(self.marines) == 0 or
                len(self.zerglings) == 0):
            return "Draw"

        # Game is still ongoing
        return "U"  # Undetermined/Unfinished

    def evaluate_position(self):
        """
        Advanced board evaluation considering multiple factors.
        """
        if self.state != "U":
            # Terminal state evaluation
            game_state = self.state
            if game_state == "Marine Wins":
                return float('inf')
            elif game_state == "Zergling Wins":
                return float('-inf')
            else:
                return 0



        def calculate_unit_score(units, enemy_units):
            total_score = 0
            for unit in units:
                unit_score = (
                        unit.hp * self.health_weight +  # Health component
                        calculate_positioning_score(unit, enemy_units) * self.positioning_weight
                )
                total_score += unit_score
            return total_score

        def calculate_positioning_score(unit, enemy_units):
            """
            Calculate a positioning score based on attack range:
            - Melee units (attack_range=1): Slightly favor being in melee range (<= 1).
            - Ranged units (attack_range>1): Slightly favor being within attack range of at least one enemy.
            """
            if not enemy_units:
                return 0

            distances = [((unit.x - enemy.x) ** 2 + (unit.y - enemy.y) ** 2) ** 0.5 for enemy in enemy_units]
            min_distance = min(distances)

            if unit.attack_range == 1:  # Melee unit
                if min_distance <= 1.0:
                    positioning_score = 0.6  # Slightly favor melee range
                else:
                    # Linearly decrease score as distance increases up to 3, but less severely
                    positioning_score = max(0.2, 0.6 - (min_distance / 5))  # Score decreases slowly up to distance 3
            else:  # Ranged unit
                if min_distance <= unit.attack_range:
                    positioning_score = 0.5  # Slightly favor being within attack range
                else:
                    positioning_score = 0.3  # Give a small incentive to be in range

            return positioning_score



        marine_score = calculate_unit_score(self.marines, self.zerglings)
        zergling_score = calculate_unit_score(self.zerglings, self.marines)

        # Positive score favors Marines, negative favors Zerglings
        evaluation = marine_score - zergling_score
        return evaluation

## Unit classes
The unit super class is used to define and hold the information of each unit type.
This holds their x,y co-ordinates and the HP of the unit, its potential damage and its movespeed (Speed).

In [13]:
class Unit:
    def __init__(self, x, y, hp, speed, damage, attack_range, team):
        self.x = x
        self.y = y
        self.hp = hp
        self.speed = speed
        self.damage = damage
        self.attack_range = attack_range
        self.team = team  # 'Marine' or 'Zerg'
        self.engaged_in_melee = False



class Marine(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (9.8)
        super().__init__(x, y, hp=45, speed=3, damage=9.8, attack_range=5, team='Marine')


class Zergling(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (10)
        super().__init__(x, y, hp=35, speed=4, damage=10, attack_range=1, team='Zergling')



In [14]:
# Create a board
board = CurrentBoard(width=30, height=10,zergling_count=6, marine_count=3)

# Initial setup will be automatic by calling setup_initial_board()
board.setup_initial_board()

# Display the initial board state
board.display_board()

# was having issues with the board set up, display and debug helped to show this better.



Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6


# Search Tree Node

There is some differences between the search tree node provided and this version. Mainly based around child generation and the min_max function, which went through several itterations.

At first there was a Best_Move function trying to determine what board would be best for the current team to make, this complicated things and was removed.

The min_max value also returned a value however this did not equate to a child on the board, as such I changed this to also return what the best move for the current team was, this was clearer than simply taking the first or last of the sorted children. As there was some confusion as to what to use depending on what team the player picked. This final version allow the return of the correct child for the team given.

A max depth was added to ensure the configuration and testing of different ply depths throughout the project.

In [15]:
class SearchTreeNode:
    def __init__(self, current_board, playing_team, depth=0):
        self.current_board = current_board
        self.playing_team = playing_team
        self.value_is_assigned = False
        self.depth = depth
        self.children = []
        self.value = None
        self.best_move = None

    def is_game_over(self):
        # Check if game is over based on board state
        game_state = self.current_board.state_of_board()
        return game_state != "U"



    def generate_children(self):
        # Generate possible move sets for the current team
        team_action_combinations = self.current_board.get_team_possible_actions(self.playing_team)
        # Create child nodes for each action combination
        for action_combination in team_action_combinations:
            # Create a deep copy of the current board
            new_board = copy.deepcopy(self.current_board)

            # Select units for the current team
            units = new_board.marines if self.playing_team == 'Marine' else new_board.zerglings

            # Apply moves to the board
            for unit, (action_type, target) in zip(units, action_combination):
                if action_type == 'move':
                    new_board.move_unit(unit, target)
                elif action_type == 'attack':
                    new_board.attack_unit(unit, target)

            # Create child node with opposite team
            child_team = 'Zergling' if self.playing_team == 'Marine' else 'Marine'
            child = SearchTreeNode(
                new_board,
                child_team,
                depth=self.depth + 1,
            )

            self.children.append(child)


    def create_board_from_action(self, unit, action):
        # Deep copy of current board
        new_board = copy.deepcopy(self.current_board)

        # Apply the action
        action_type, target = action

        if action_type == 'move':
            # Move unit to new position
            new_board.move_unit(unit, target)
        elif action_type == 'attack':
            # Perform attack
            new_board.attack_unit(unit, target)

        return new_board

    def min_max_value(self, current_depth=0, max_depth=3):
        # If value is already assigned, return it
        if hasattr(self, 'value_is_assigned') and self.value_is_assigned:
            return self.value, None  # Return value and no best child

        # Check if reached max depth or game is over
        # Use current_depth to track actual depth, not self.depth
        if current_depth >= max_depth or self.is_game_over():
            evaluation = self.current_board.evaluate_position()
            return evaluation, None  # Return evaluation and no best child

        # Ensure children are generated
        if not self.children:
            self.generate_children()

        # If still no children, return evaluation
        if not self.children:
            evaluation = self.current_board.evaluate_position()
            return evaluation, None  # Return evaluation and no best child

        # Compute values for children recursively
        child_values = []
        for child in self.children:
            # Pass incremented depth to track recursion depth
            child_value, _ = child.min_max_value(current_depth + 1, max_depth)  # Discard child's best child
            child_values.append(child_value)
            child.value = child_value

        # Select value based on current playing team
        if self.playing_team == 'Marine':
            # Marine tries to maximize, so take the highest value
            if child_values:
                self.value = max(child_values)
                best_child_index = child_values.index(self.value)
                best_child = self.children[best_child_index]
            else:
                self.value = self.current_board.evaluate_position()
                best_child = None
        else:
            # Zerg tries to minimize, so take the lowest value
            if child_values:
                self.value = min(child_values)
                best_child_index = child_values.index(self.value)
                best_child = self.children[best_child_index]
            else:
                self.value = self.current_board.evaluate_position()
                best_child = None

        # Mark value as assigned
        self.value_is_assigned = True
        self.children = sorted(self.children, key=lambda x: x.value)

        return self.value, best_child




In [16]:
search_tree = SearchTreeNode(board, 'Zergling')


In [17]:
search_tree.current_board.display_board()


Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6


In [18]:
best_move, best_child = search_tree.min_max_value(max_depth=2)

In [19]:
search_tree.children[0].value

-30.090000000000003

In [20]:
search_tree.children[-1].value


-30.090000000000003

In [21]:
len(search_tree.children)

324

In [22]:
best_move

-30.090000000000003

In [23]:
search_tree.children[0].current_board.display_board()


Debug - Unit Positions:
Zergling: x=4, y=0 hp=35
Zergling: x=4, y=1 hp=35
Zergling: x=4, y=2 hp=35
Zergling: x=5, y=0 hp=35
Zergling: x=5, y=5 hp=35
Zergling: x=5, y=2 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . Z Z . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . Z . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . Z Z . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 . . . . . Z . . . . . . . . . . . . . . . . . . . . . . M M
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6


# Play Function

The play function was based off the tic-tac toe example, however, the key change is that there is no user input after the first screen, the player choses a side and then the game is auto played based in the inputs, max_ply and intial map and unit counts are added as a varaible.

To play the game you need to enter Z or M to choose your side and press the okay button.

One issue I had was that I had incorrectly set the player_side to = Zerg if Z was selected. This caused some unexpected behaviour where the enemy side was also acting as the Zerg and marines did not move at all, but if you picked marines it would work as expected.

The players_turn variable is also set to false on both choices, this means the enemy will always start.

In [24]:
def play_StarCraft(width=30, height=10, marine_count=4, zergling_count=8,max_ply=2):
    # Ask player to choose starting side
    while True:
        try:
            player_side = input("Choose your side (M for Marine, Z for Zerg): ").upper().strip()

            if player_side == 'M':
                player_side = 'Marine'
                break
            elif player_side == 'Z':
                # had this as Zerg for ages, caused a lot of issues....
                player_side = 'Zergling'
                break
            else:
                print("Invalid input. Please enter 'M' for Marine or 'Z' for Zerg.")
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Please try again.")

    # Create initial board with specified parameters
    cb = CurrentBoard(width, height, marine_count, zergling_count)
    cb.setup_initial_board()

    # Determine who starts based on player's choice
    if player_side == 'Marine':
        current_player = 'Zergling'  # AI will start with Zerg move
        players_turn = False
    else:
        current_player = 'Marine'
        players_turn = False

    # Game loop
    for _ in range(100):  # Prevent infinite loop
        # Display current board state
        cb.display_board()

        # Prompt to continue
        # input(f"Press Enter to continue {current_player}'s turn...")

        if players_turn:
            # Player's turn
            search_tree = SearchTreeNode(cb, player_side)
            best_value, best_child = search_tree.min_max_value(max_depth=max_ply)
            cb = best_child.current_board
        else:
            # AI's turn
            search_tree = SearchTreeNode(cb, current_player)
            best_value, best_child = search_tree.min_max_value(max_depth=max_ply)
            cb = best_child.current_board



        # Check game state
        game_state = cb.state_of_board()
        if game_state != "U":
            cb.display_board()
            if game_state == "D":
                print("Draw!")
            else:
                print(f"{game_state} wins!")
            break

        # Switch players
        current_player = 'Zergling' if current_player == 'Marine' else 'Marine'
        players_turn = not players_turn

    print("Game Over, if no winner announced above, Draw")


In [25]:
play_StarCraft()


Debug - Unit Positions:
Zergling: x=0, y=3 hp=35
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=3 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=29, y=4 hp=45
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 4, Zerglings: 8

Debug - Unit Positions:
Zergling: x=0, y=3

KeyboardInterrupt: 

In [34]:
play_StarCraft(marine_count=3, zergling_count=4,max_ply=2)



Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 4

Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Marine: x=26, y=5 hp=45
Marine: x=25, y=4 

# Testing & Evaluation

As stated in the board discussion, just getting the baord to work and the game to function took many itterations, the cases where game pieces ended up in areas where no move could be possible is one itteration not mentioned previously.

This occured when one itteration of possible actions did not have a default move possibility, this was resolved with the retreat function which allowed retreat or stay still as a last resort.

One key itteration was about the movement of the zerglings, even though their intention was to surround the enemy, if they were allowed to retreat, they would always keep one going back to keep the overall hp of the team higher, however this meant that the marines were able to pick off the zergs one by one each time, removal of this allowed retreat for the zergs was a key point to their epected behavior.

The ability for the direction to change based on the positioning of the enemy units was also essential to reducing branching factor but still have realistic variations of the game.

# Results and Conclusion

The entirety of this project has been spent trying to get the board and evaluation working correctly, and the main focus has been around choosing the correct movement patterns and attack options for units, this went through many itterations especially in terms of reducing branching factor and checking the expected behavior of the units.

The use of the search tree to effectively choose a move that would be beneficial is an interesting concept for any baord game, the objective of the evaluation is the act of checking all possible moves x steps in the future and picking the one that would be most beneficial to the player at that time.

One interesting occurrence is that in the desired game board of 8 zergs and 4 marines, there is the possibility of a draw conidtion, where one unit of each remains.

![image](./images/draw.png)

Possible further improvements not explored, is the previously mentioned evaluation weighting based on percent of units remaining. This should improve win rates of the marines, since losing a unit is not the same as losing hp on multiple units.

Currently, the weighting is just based on unit hp, therefore the score is not proportional to the overall team, rather just a summation of the units hp multiplied by the weighting factor.